# Replicating Results


### What is this?
This notebook provides step-by-step instructions for replicating the results for multi-adjustment

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

import os
import yaml
import sqlalchemy

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = None

from IPython.display import display 
from itertools import permutations, combinations
from jinja2 import Template
import dateparser


from ohio.ext.numpy import pg_copy_to_table

Specify The Database, demo_col of interest, demo_values of interest and the procedure you wish to use for adjustment

In [2]:
database="dhananja_donors"

In [3]:
working_schema, results_schema = "bias_working", "bias_results"

In [4]:
demo_col = "plevel"

In [5]:
demo_values = ["not_highest", "highest"]

## Getting Set Up

You will need a database with the following schemas:
- **public**: The raw data from donors choose as well as some tables with calculated features and intermediate modeling tables
- **model_metadata**: Information about the models we ran, such as model types and hyperparameters (models were run with `triage`, which generates this schema. In `triage` a "model group" specifies a type of model and associated hyperparameter values, while a "model" is an instantiation of a given model group on a specific temporal validation split). Note that this schema contains information on other model runs with this dataset, in addition to the run used for the current study of fairness-accuracy trade-offs.
- **test_results**: Validation set statistics and predictions for the models. Here, `test_results.predictions` contains project-level predicted scores from each model in the grid, while `test_results.evaluations` contains aggregated summary statistics for each model.
- **train_results**: Training set statistics for the models, including feature importances.
- **features**: Intermediate tables containing calculated features from the `triage` run.
- **bias_working**: Intermediate tables from the bias analysis, as well as the mapping table between projects and school poverty levels, `bias_working.entity_demos`.
- **bias_results_submitted**: Results of the fairness-accuracy trade-offs from the study as submitted (see below to use these to replicate the figures from the study).
- **bias_results**: Empty bias analysis results tables that will be populated by re-running the fairness adjustments (see below for instructions).


Finally, to connect to the database, you'll need a yaml file named `db_profile.yaml` in the same directory as this notebook with your connection info:
```yaml
host: {POSTGRES_HOST}
user: {POSTGRES_USER}
db: education_crowdfunding
pass: {POSTGRES_PASSWORD}
port: {POSTGRES_PORT}
```


In [12]:
with open('../../config/db_default_profile.yaml') as fd:
    config = yaml.full_load(fd)
    dburl = sqlalchemy.engine.url.URL(
        "postgresql",
        host=config["host"],
        username=config["user"],
        database=database,
        password=config["pass"],
        port=config["port"],
    )
    engine = sqlalchemy.create_engine(dburl, poolclass=sqlalchemy.pool.QueuePool)

/tmp/ipykernel_27940/127057796.py:3: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  dburl = sqlalchemy.engine.url.URL(


In [90]:
def get_frac_demos(model_group_id=None, train_end_time=None):
    if model_group_id is None and train_end_time is None:
        sel_string = ""
    elif model_group_id is None:
        sel_string = f"and train_end_time = '{train_end_time}'"
    elif train_end_time is None:
        sel_string = f"and model_group_id = {model_group_id}"
    else:
        sel_string = f"and train_end_time = '{train_end_time}' and model_group_id = {model_group_id}"
    sql = f"""
    select model_group_id, train_end_time, weight, count(distinct entity_id) 
    from {results_schema}.selected_entities se natural join {working_schema}.entity_demos ed
    where ed.{demo_col} = '{demo_values[0]}' {sel_string}
    group by 1, 2, 3 
    """
    ts_df = pd.read_sql(sql, engine)
    return ts_df

In [97]:
def get_difference(weight1, weight2, model_group_id=None, train_end_time=None):
    if model_group_id is None and train_end_time is None:
        sel_string = f""
    elif model_group_id is None:
        sel_string = f"WHERE train_end_time = '{train_end_time}'"
    elif train_end_time is None:
        sel_string = f"WHERE model_group_id = {model_group_id}"
    else:
        sel_string = f"WHERE model_group_id = {model_group_id} and train_end_time = '{train_end_time}'"
    
    sql = f"""
        with all_entities as (select * 
        from {results_schema}.selected_entities se natural join {working_schema}.entity_demos ed)
        , e1 as (select * from all_entities where weight = {weight1})
        , e2 as (select * from all_entities where weight = {weight2})
        , e3 as (select * from e1 where (model_group_id , train_end_time , entity_id) not in (select model_group_id , train_end_time , entity_id  from e2))
        , e4 as (select * from e2 where (model_group_id , train_end_time , entity_id) not in (select model_group_id , train_end_time , entity_id  from e1))
        SELECT distinct * from e3 {sel_string} UNION ALL SELECT distinct * FROM e4 {sel_string}
        """    
    ts_df = pd.read_sql(sql, engine)
    return ts_df

In [98]:
get_frac_demos()


    select model_group_id, train_end_time, weight, count(distinct entity_id) 
    from bias_results.selected_entities se natural join bias_working.entity_demos ed
    where ed.plevel = 'not_highest' 
    group by 1, 2, 3 
    


,model_group_id,train_end_time,weight,count
0,915,2011-03-01,0.01,430
1,915,2011-03-01,0.10,430
2,915,2011-03-01,0.20,430
3,915,2011-03-01,0.30,430
4,915,2011-03-01,0.40,430
...,...,...,...,...
971,994,2013-01-01,0.60,498
972,994,2013-01-01,0.70,500
973,994,2013-01-01,0.80,502
974,994,2013-01-01,0.90,504


In [99]:
diff_df = get_difference(weight1=0.9, weight2=0.8)

In [100]:
print(f"Model Group IDs Used")
print(diff_df['model_group_id'].unique())
print(f"Train End Times Used")
print(diff_df['train_end_time'].unique())

Model Group IDs Used
[942 994 928 925 965 918 915 960]
Train End Times Used
['2012-09-01T00:00:00.000000000' '2011-09-01T00:00:00.000000000'
 '2011-11-01T00:00:00.000000000' '2012-05-01T00:00:00.000000000'
 '2012-11-01T00:00:00.000000000' '2012-03-01T00:00:00.000000000'
 '2012-01-01T00:00:00.000000000' '2013-01-01T00:00:00.000000000'
 '2012-07-01T00:00:00.000000000' '2011-07-01T00:00:00.000000000'
 '2011-05-01T00:00:00.000000000' '2011-03-01T00:00:00.000000000']


In [101]:
model_group_id = 915
train_end_time = "2012-03-01"

In [102]:
get_frac_demos(model_group_id=model_group_id, train_end_time=train_end_time)


    select model_group_id, train_end_time, weight, count(distinct entity_id) 
    from bias_results.selected_entities se natural join bias_working.entity_demos ed
    where ed.plevel = 'not_highest' and train_end_time = '2012-03-01' and model_group_id = 915
    group by 1, 2, 3 
    


,model_group_id,train_end_time,weight,count
0,915,2012-03-01,0.01,478
1,915,2012-03-01,0.10,477
2,915,2012-03-01,0.20,476
3,915,2012-03-01,0.30,475
4,915,2012-03-01,0.40,474
5,915,2012-03-01,0.50,473
6,915,2012-03-01,0.60,471
7,915,2012-03-01,0.70,470
8,915,2012-03-01,0.80,469
9,915,2012-03-01,0.90,468


In [103]:
get_difference(weight1=0.9, weight2=0.8, model_group_id=model_group_id, train_end_time=train_end_time)

,entity_id,model_group_id,train_end_time,score,label_value,model_rank,rn_demo,weight,as_of_date,plevel
0,103676,915,2012-03-01,0.64011,1,1126,532,0.9,2012-08-13,highest
1,372556,915,2012-03-01,0.66103,1,904,469,0.8,2012-06-25,not_highest


# Performing Entity Analysis

Assumes that the model_adjustment_results_{demo_col} table is populated correctly for model selection

In [ ]:
from DJRecallAdjuster import entity_analysis_procedure

In [1]:
entity_analysis_procedure(pause_phases=False)

NameError: name 'ra_procedure' is not defined